# Lab 4

## Init data

![Task](image.png)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate as tb

X = np.array([i+1 for i in range(5)])
Y = np.array([2 + 0.5 * i for i in range(5)])

lenData = len(X)
nan = np.nan
fmt_float = '.3f'

Data = np.array(
    [
        [nan, nan, nan, 3, 4],
        [nan, nan, 5, 3, 5],
        [nan, 3, 6, 4, nan],
        [4, 4, 2, 2, nan],
        [1, 4, nan, nan, nan]
    ]
)

ny_SumsOfYInData = np.array([np.nansum(Data[i, :]) for i in range(lenData)])
nx_SumsOfYInData = np.array([np.nansum(Data[:, i]) for i in range(lenData)])


def tabulate(a: np.ndarray, headers: np.ndarray = []):
    print(tb(np.array([np.round(i, 3) for i in a]), headers=headers,
          tablefmt='rounded_grid', numalign='center'))


In [ ]:
tabulate(Data)


## Correlation field

In [ ]:
# trend line
start = (np.flip(X)[0] + np.flip(Y)[0]) / 2
notnanData = (((np.flip(X)[-1] + np.flip(Y)[-1]) / 2) - start) / lenData

tl = [start + notnanData * i for i in range(lenData)]

notnanData = [[[], [], []] for i in range(lenData)]

for j in range(lenData):
    for i in range(lenData):
        if (np.isnan(Data[i, j])):
            continue
        notnanData[i][0].append(i + 1)
        notnanData[i][1].append(Y[j])
        notnanData[i][2].append(Data[j, i])

plt.plot(np.flip(Y), X, label='Y / X')
plt.plot(np.flip(X), Y, label='X / Y')
plt.plot(tl, Y, label='Trend line', c='gray', ls='--')
for i in range(lenData):
    plt.scatter(notnanData[i][0], notnanData[i][1], c='black')
plt.title('Correlation field')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend(loc='lower left')


## Least squares regression line equation

### Plot

In [ ]:
y_avarage_xj = np.zeros(lenData)

for j in range(lenData):
    q = np.sum(notnanData[j][-1])* Y[j]
    y_avarage_xj[j] = q / nx_SumsOfYInData[j]

plt.scatter(X, y_avarage_xj, c="black")
plt.plot(X, Y, label='X on Y')
plt.legend()

### The least squares method

In [ ]:
# TODO: task 2